In [ ]:
import keras
from keras.preprocessing import image
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model = VGG16(weights='imagenet')

In [ ]:
# The local path to our target image
img_path = '/tmp/workspace/Pictures/elephants.jpg'
#img_path = './cat_1700.jpg'

# `img` is a PIL image of size 224x224
img = image.load_img(img_path, target_size=(224, 224))

# `x` is a float32 Numpy array of shape (224, 224, 3)
x = image.img_to_array(img)

# We add a dimension to transform our array into a "batch"
# of size (1, 224, 224, 3)
x = np.expand_dims(x, axis=0)

# Finally we preprocess the batch
# (this does channel-wise color normalization)
x = preprocess_input(x)
img

In [ ]:
preds = model.predict(x)
maxclass = np.argmax(preds[0])
print('Predicted:', decode_predictions(preds, top=3)[0])
print('Class with maximum score:', maxclass)
print('Score: ', preds[0][maxclass])

In [ ]:
# This is the entry in the prediction vector with maximum score
maxclass_output = model.output[:, maxclass]

# The is the output feature map of the `block5_conv3` layer,
# the last convolutional layer in VGG16
last_conv_layer = model.get_layer('block5_conv3')

# This is the gradient of the "wallaby" class with regard to
# the output feature map of `block5_conv3`
grads = K.gradients(maxclass_output, last_conv_layer.output)[0]

# This is a vector of shape (512,), where each entry
# is the mean intensity of the gradient over a specific feature map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# This function allows us to access the values of the quantities we just defined:
# `pooled_grads` and the output feature map of `block5_conv3`,
# given a sample image
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

# These are the values of these two quantities, as Numpy arrays,
# given our sample image
pooled_grads_value, conv_layer_output_value = iterate([x])

# We multiply each channel in the feature map array
# by "how important this channel is" with regard to the wallaby class
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# The channel-wise mean of the resulting feature map
# is our heatmap of class activation
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()
plt.imshow(img)
plt.show()